### Gemini API

In [ ]:
import streamlit as st
import google.generativeai as genai
API = "API_KEY"
#API key
genai.configure(api_key=API)

#Loading the gemini model
model = genai.GenerativeModel("models/gemini-1.5-flash")

# Setting up the page for Assistant
st.set_page_config(page_title="Gemini Travel Assistant", page_icon="🧳")
st.title("🌍 Personalized Travel Itinerary")
st.markdown("Let AI help you plan a perfect trip tailored to your destination, days, and budget.")

# 📥 User inputs
name = st.text_input("👤 What is your name?")
origin = st.text_input("🌍 You want to start your journey from:")
type_of_trip = st.selectbox("🧳 What kind of trip are you looking for?", ["", "Solo", "Friends", "Couple", "Family"], index=0)
destination = st.text_input("📌 Enter your travel destination:")
days = st.slider("📆 Number of days", min_value=1, max_value=20, value=5)
budget = st.selectbox("💸 What is your budget for the trip (in USD)?", ["", "< 500", "500-1000", "1000-2000", "> 2000"])
diet_options = st.selectbox("🍽️ What kind of diet do you prefer?", ["", "vegetarian", "non-vegetarian", "vegan", "mix"])
people = st.text_input("👨‍👩‍👧 How many people are travelling (including you)?")
stay_options = st.selectbox("🏨 Preferred stay option:", ["", "Airbnb", "Home stay", "Hotel", "Hostels", "Bunkers"])

# Creating a button for getting travel plan
if st.button("Create a Travel Plan"):
    if "" in [name, origin, destination, type_of_trip, budget, diet_options, people, stay_options]:
        st.warning("Please fill in all fields before generating the itinerary.")
    else:
        with st.spinner("Generating your personalized itinerary..."):

            #Setting up a prompt
            prompt = f"""
            Hi I need a travel assistance. My name is {name} and I want to have a {type_of_trip.lower()} trip from {origin} to {destination}.
            I will share the trip details with you:
            - Duration of the trip: {days} days
            - Total number of travelers: {people}
            - Total budget: {budget} USD
            - Kind of diet in the trip: {diet_options}
            - Preferred stay: {stay_options}

            Create a daily itinerary that includes:
            - Morning, afternoon, and evening plans
            - Local attractions, experiences, and culture
            - Underrated and budget-friendly travel tips
            - Accommodation and transport suggestions
            - Include local dishes and best places to eat food
            - Ensure everything fits the budget

            Write in a clean, informative, and friendly tone.
            """

            # Get Gemini's response
            response = model.generate_content(prompt)
            plan = response.text

            # Show output in structured way
            st.subheader("🗺️  Travel Itenary")
            st.markdown(plan)

            # Iteniary can be downloaded as PDF
            import base64
            def get_download_link(text, filename="itinerary.txt"):
                b64 = base64.b64encode(text.encode()).decode()
                return f'<a href="data:file/txt;base64,{b64}" download="{filename}">📥 Download Itinerary</a>'

            st.markdown(get_download_link(plan), unsafe_allow_html=True)


## GPT2 and microsoft phi-1_5

In [1]:
#pip install tensorflow

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,GPT2Tokenizer, GPT2LMHeadModel

/Users/dhananjaymishra/Downloads/MQ/Semester4/COMP8420-Advance NLP/assignment2/travel_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Creting a fucntion to call the output
def run_model(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        do_sample=True,
        temperature=0.8,
        top_k=50,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [5]:
#Loading Gpt-2 LLM locally
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()



# We will use 5 different prompt types for the LLM of gpt2 and microfsoft phi-1_5, we will now see their response
prompt_dict = {
    "Peaceful": "I’m tired of city chaos. Suggest a 7-day peaceful nature escape in the mountains of Himalayas with minimal crowds.",
    "Romantic": "I want to plan a 5-day romantic getaway in Venice, Italy for me and my partner. Include candlelight dinners and gondola rides.",
    "Adventure": "Plan a 10-day thrill-packed solo adventure in South America, including hiking, rafting, and offbeat places.",
    "Budget": "I have a very tight budget. Suggest a 7-day trip to Southeast Asia under $500, with cheap food, hostels, and cultural experiences.",
    "Foodie": "I'm a vegetarian food lover. Recommend a 6-day food-focused tour across South India — must include street food and hidden local restaurants."
}

print("GPT-2 Local Responses:\n" + "="*40)
output_gpt = {} #Getting the response in a dictionary

for label, text in prompt_dict.items():
    response = run_model(text)
    output_gpt[label] = response
    print(f"\{label} Prompt:\n{text}\n")
    print(f"GPT-2 Response:\n{response}\n{'-'*40}")




The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


GPT-2 Local Responses:
\Peaceful Prompt:
I’m tired of city chaos. Suggest a 7-day peaceful nature escape in the mountains of Himalayas with minimal crowds.

GPT-2 Response:
I’m tired of city chaos. Suggest a 7-day peaceful nature escape in the mountains of Himalayas with minimal crowds.

I didn't want to be the girl in the park. Like everyone else, I wanted to find a good way to share my adventures. So I decided to write a story in a different language.

To start writing, I used a little Python script to create a "playbook."

import playbook from pygame import Playbook
----------------------------------------
\Romantic Prompt:
I want to plan a 5-day romantic getaway in Venice, Italy for me and my partner. Include candlelight dinners and gondola rides.

GPT-2 Response:
I want to plan a 5-day romantic getaway in Venice, Italy for me and my partner. Include candlelight dinners and gondola rides.

We have an amazing place to stay in Venice, Italy, but we do not have a hotel booked from Ven

In [6]:


model_id = "microsoft/phi-1_5"

# Load tokenizer and model for microfsoft phi-1_5
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16
)


# Display output clearly
print("microfsoft phi-1_5 Local Responses:\n" + "="*40)
output_gpt = {}

for label, text in prompt_dict.items():
    response = run_model(text)
    output_gpt[label] = response
    print(f"\n{label} Prompt:\n{text}\n")
    print(f"microfsoft phi-1_5 Response:\n{response}\n{'-'*40}")



microfsoft phi-1_5 Local Responses:

Peaceful Prompt:
I’m tired of city chaos. Suggest a 7-day peaceful nature escape in the mountains of Himalayas with minimal crowds.

microfsoft phi-1_5 Response:
I’m tired of city chaos. Suggest a 7-day peaceful nature escape in the mountains of Himalayas with minimal crowds.

Alex: That sounds great! I've always wanted to visit the Himalayas. The idea of a 7-day peaceful nature escape in the mountains... It seems like the perfect getaway.

David: Absolutely, nature has a way of rejuvenating our souls. By the way, did you know that the Great Barrier Reef is the largest
----------------------------------------

Romantic Prompt:
I want to plan a 5-day romantic getaway in Venice, Italy for me and my partner. Include candlelight dinners and gondola rides.

microfsoft phi-1_5 Response:
I want to plan a 5-day romantic getaway in Venice, Italy for me and my partner. Include candlelight dinners and gondola rides.

Linda: That sounds perfect. What airline sh